In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import csv
!pip install textblob
!pip install contractions
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('brown')
nltk.download('punkt')
from textblob import TextBlob
import re
from tqdm.notebook import tqdm
import contractions
tqdm.pandas()

     |████████████████████████████████| 283 kB 5.4 MB/s 
     |████████████████████████████████| 321 kB 48.6 MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85450 sha256=c5a9b74119c43c024e4f9876af8bf568aadb5f09c1a70fa98149b768655b7cdd
  Stored in directory: /root/.cache/pip/wheels/25/19/a6/8f363d9939162782bb8439d886469756271abc01f76fbd790f
Successfully built pyahocorasick
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# data = pd.read_csv('/content/drive/MyDrive/Amazon ML Challenge 2021/dataset/train.csv', escapechar="\\", quoting=csv.QUOTE_NONE)
# all_na = list(data[data['TITLE'].isna()][data['DESCRIPTION'].isna()][data['BULLET_POINTS'].isna()][data['BRAND'].isna()].index)
# data.drop(all_na,inplace=True)
# data.reset_index(inplace=True,drop=False)

In [ ]:
# data.sample()

In [ ]:
data = pd.read_pickle("/content/drive/MyDrive/Amazon ML Challenge 2021/dataset/train_keys.pkl",compression="zip")

In [ ]:
pd.set_option('max_colwidth', -1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


In [ ]:
data['BROWSE_NODE_ID'].value_counts().head(10)

1045    215698
5       70318 
1251    51929 
1052    45553 
4       34169 
1295    32040 
1024    31961 
374     31896 
773     25899 
1194    25893 
Name: BROWSE_NODE_ID, dtype: int64

In [ ]:
df=data[data['BROWSE_NODE_ID'] == 374]
df.sample(10)

,index,TITLE,DESCRIPTION,BULLET_POINTS,BRAND,BROWSE_NODE_ID,keywords
2132275,2132281,cassby anti glare scratch proof tempered glass for oppo a5 2020,cassby premium glass protectors are the latest in state of the art screen protection technology highly durable and scratch resistant chip resistant this strong 9h hardness level protector will provide your cell phone the best protection against drops bumps scratches and normal wear and tear get all the protection without any bulk cassby glass protectors are a mere 33mm thin making them ultra light weight to allow for a delicate touch style screen protector that promises not to interfere with the sensitivity of your touchscreen to top it all off the cassby glass protectors are designed to be 99 99 transparent to promote an optimal natural crystal clear viewing experience,oppo a5 2020 specifically designed with high quality long lasting film featuring exceptional display clarity full screen coverage edge to edge protection protection premium quality agc glass materials protects your phone screen from daily wear and tear scratches scrapes and explosion which reach 9h hardness military shield 3 times stronger than common glass harder than a knife anti shatter if broken the glass will break into small parts but stay in one place making it safer for glass screen of your phone easy to install dust free fingerprint free one push super easy installation bubble free high responsive precise laser cutout and delivers full touchscreen sensitivity original optical clarity and touch screen sensitivity,cassby,374,tear scratches scrapes | free fingerprint | glare scratch proof | exceptional display clarity | military shield | touch screen sensitivity | small parts | install dust | original optical clarity | art screen protection technology | free high responsive precise laser cutout | oppo a5 | high quality | cassby glass protectors | common glass | light weight | resistant chip | cassby premium glass protectors | bulk cassby glass protectors | 9h hardness | place making | strong 9h hardness level protector | cell phone | full touchscreen sensitivity | edge protection protection premium quality agc glass materials protects | phone screen | delicate touch style screen protector | drops bumps scratches | glass screen | natural crystal | super easy installation bubble | mere 33mm | full screen coverage edge
2517512,2517519,trink premium high quality mobile lamination roll anti scratch and dust proof 2 layer 2 meter,trink premium hi quality 3 layer lamination roll for mobiles laptop crystal clear film lamination roll sheet lamination film screen guard for mobile tablet laptop 12 inch x 70 inch of ulove brand lamination provides superior defence against elements of on the go life scratches dust dirt anti scratch and dust proof anti glare and uv quality of the product and customer satisfaction is 100 guaranteed removal without residuary mark extra thick films protects hand set from damagehi quality 3 layer lamination roll for mobiles laptop crystal clear film lamination roll sheet lamination film screen guard for mobile tablet laptop 12 inch x 70 inch of ulove brand lamination provides superior defence against elements of on the go life scratches dust dirt anti scratch and dust proof anti glare and uv quality of the product and customer satisfaction is 100 guaranteed removal without residuary mark extra thick films protects hand set from damage hi quality 3 layer lamination roll for mobiles laptop crystal clear film lamination roll sheet lamination film screen guard for mobile tablet laptop 12 inch x 70 inch of ulove brand lamination provides superior defence against elements of on the go life scratches dust dirt anti scratch and dust proof anti glare and uv quality of the product and customer satisfaction is 100 guaranteed removal without residuary mark extra thick films protects hand set from damage,cristal clear film main tains screen s brightness extra thick films protects hand set from damage shock

In [ ]:
df['BRAND'].value_counts()

clovia                 1850
sona                   1243
marks spencer          822 
enamor                 801 
amante                 748 
                      ...  
paras                  1   
esteem                 1   
agent provocateur      1   
your shopping store    1   
alpha man              1   
Name: BRAND, Length: 1474, dtype: int64

In [ ]:
data[data['BRAND'] == 'max']

,index,TITLE,DESCRIPTION,BULLET_POINTS,BRAND,BROWSE_NODE_ID,keywords
9895,9895,max boy s plain regular fit t shirt w18dmr10 grey 2 3 y,NaN,color name grey material cotton long sleeve machine wash,max,1003,long sleeve machine | color name grey material cotton | max boy s | plain regular fit t shirt w18dmr10 grey
9907,9907,max men s printed slim fit t shirt catpa1871kty grey l,NaN,color name grey material cotton long sleeve round neck regular fit machine wash,max,5,max men s | color name grey material cotton | slim fit t shirt catpa1871kty grey l | long sleeve round neck | regular fit machine
9908,9908,max men s checkered slim fit casual shirt inchwi1807fs khaki xl,NaN,color name khaki material cotton long sleeve regular fit classic collar machine wash,max,1024,max men s | color name khaki material cotton | regular fit classic collar machine | slim fit casual shirt inchwi1807fs khaki xl | long sleeve
9918,9918,max baby girl s slim fit jeans sufd09gx mid blue 12 18 months,NaN,color name mid blue material cotton machine wash,max,9918,color name mid | slim fit jeans sufd09gx mid | blue material cotton machine | max baby girl s
9928,9928,max men s striped slim fit polo bsppa1803kts red m,NaN,color name red material cotton long sleeve regular fit machine wash,max,1213,max men s | color name | red material cotton | red m | slim fit polo bsppa1803kts | long sleeve | regular fit machine
...,...,...,...,...,...,...,...
2901267,2901276,max women s regular fit top sp20yfp30mint mint x small,NaN,color name mint material cotton machine wash body blouse cap sleeve,max,1194,body blouse cap sleeve | color name mint material cotton machine | max women s | regular fit top sp20yfp30mint mint x
2902525,2902534,max polyester skater dress,NaN,color name maroon other machine wash skater midi,max,1724,color name maroon | max polyester skater dress | skater midi
2902526,2902535,max women s cotton parallel bottom scptsp20bo white large,NaN,max women solid elasticated straight pants material cotton color white design solid care instruction hand wash only closure drawstring closure,max,872,max women s cotton parallel bottom scptsp20bo | straight pants material cotton color | white design | max women | solid care instruction hand
2902541,2902550,max regular fit t shirt design 20 pack of 2,NaN,printed t shirt cotton fabric made in india half sleeves,max,1049,regular fit t shirt design | t shirt cotton fabric


In [ ]:
def generic_preprocessor(text):
    if text!=text:
        return text
    text = str(text)
    soup = BeautifulSoup(text)
    text = soup.get_text()
    text = text.encode('ascii', 'ignore')
    text = text.decode()
    text = text.lower()
    # text = str(TextBlob(text).correct())
    text = contractions.fix(text)
    text = re.sub('[^a-zA-Z0-9]', ' ', text)
    text = ' '.join(text.split())
    return text

In [ ]:
# data["TITLE"] = data["TITLE"].progress_apply(generic_preprocessor)
# data.to_pickle("/content/drive/MyDrive/Amazon ML Challenge 2021/dataset/train.pkl",compression="zip")

# data["DESCRIPTION"] = data["DESCRIPTION"].progress_apply(generic_preprocessor)
# data.to_pickle("/content/drive/MyDrive/Amazon ML Challenge 2021/dataset/train.pkl",compression="zip")

# data["BULLET_POINTS"] = data["BULLET_POINTS"].progress_apply(generic_preprocessor)
# data.to_pickle("/content/drive/MyDrive/Amazon ML Challenge 2021/dataset/train.pkl",compression="zip")

# data["BRAND"] = data["BRAND"].progress_apply(generic_preprocessor)
# data.to_pickle("/content/drive/MyDrive/Amazon ML Challenge 2021/dataset/train.pkl",compression="zip")

/usr/local/lib/python3.7/dist-packages/bs4/__init__.py:273: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
/usr/local/lib/python3.7/dist-packages/bs4/__init__.py:336: UserWarning: "https://i.postimg.cc/pdDzGnQR/size-chart-all-tshirts.jpg" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/usr/local/lib/python3.7/dist-packages/bs4/__init__.py:336: UserWarning: "https://imgur.com/5YILUva" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/usr/local/lib/python3.7/dist-packages/bs4/__init__.py:27

/usr/local/lib/python3.7/dist-packages/bs4/__init__.py:273: UserWarning: "b'drive'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
/usr/local/lib/python3.7/dist-packages/bs4/__init__.py:336: UserWarning: "https://www.jangadi.com" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


In [4]:
data = pd.read_pickle("/content/drive/MyDrive/Amazon ML Challenge 2021/dataset/train_limited_brands.pkl",compression="zip")
data["BRAND"].fillna("OTHER_BRANDS",inplace=True)
data.dropna(inplace=True)

In [ ]:
def process(title,des,keywords,brand):
    final_keys = []
    if title == title:
         final_keys.extend(title)

    if keywords == keywords:
        final_keys.extend(keywords)

    if brand == brand:
        final_keys.extend(brand)

    final_keys = list(set(final_keys))
    return " | ".join(final_keys)

In [5]:
data["keywords"] = data["TITLE"] + " | " + data["DESCRIPTION"] + " | "+ data["BRAND"]

In [6]:
data.to_pickle("/content/drive/MyDrive/Amazon ML Challenge 2021/dataset/new-nokeys.pkl",compression="zip")